In [ ]:
import pandas
import numpy
from tqdm import tqdm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

df_anime = pandas.read_csv('top_and_bottom_anime.csv')
df_anime = df_anime.drop(columns='Id')
df_anime = df_anime.drop(columns='Rating')
df_anime = df_anime.drop(columns='Demographic')

new = pandas.DataFrame()
le = preprocessing.LabelEncoder()
for i in range(df_anime.shape[1]):
    temp = df_anime.iloc[:,i].values.tolist()
    temp = le.fit_transform(temp)
    new[df_anime.columns[i]] = temp

df_predictions = knn_predictions['Id'].copy()
df_anime_y = pandas.read_csv('anime_matrix_true.csv')
rows = df_anime_y[['Id']].copy()

#make adaboost predictions
columns = df_anime_y.columns
predictions = []

avg = []
for i in tqdm(range(1,len(columns))):   #skip first item b/c it's anime Id
    x = new
    y = df_anime_y[columns[i]]
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=42)
    
    #create cross training split for building the models
    X_cross_train, X_cross_test, y_cross_train, y_cross_test = train_test_split(x_train, y_train, test_size=0.2, random_state=1080)

    adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    adaboost.fit(X_cross_train,y_cross_train)
    avg.append(adaboost.score(X_cross_test,y_cross_test))
    predictions.append(list(adaboost.predict(X_cross_test)))

import warnings
warnings.simplefilter(action='ignore', category=pandas.errors.PerformanceWarning)
for i in tqdm(range(len(columns))):
    df_predictions[columns[i]] = predictions[i]

adaboost_predictions = df_predictions.copy()